# Effect of Parallelization

In [8]:
from utils.pipelines import *
from utils.calibration import *
from utils.helpers import *
import time
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
#Merged News data
data = read_file("./data/processed/final_df.csv")
#Doge price data
doge_price = read_file("./data/clean/doge_price.parquet").rename({"Date":"date"}, axis = 1)
#Sentiment data is the tesla news sentiment data
sentiment_data = data[data["Source"] == "Crypto_News"]

merged_df = process_and_merge_data_continous_price(sentiment_data, doge_price)

#Calculating log returns and then inspecting the return dataframe 
intermediate_data = calculate_log_returns(merged_df,'Open')
intermediate_data.head()

c:\Users\Mert\OneDrive\Desktop\Courses\FBG_Org\financial-big-data-project\utils\helpers.py:63: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  sentiment_data['date'] = sentiment_data['date'].dt.ceil('T')  # 'T' stands for minute
c:\Users\Mert\OneDrive\Desktop\Courses\FBG_Org\financial-big-data-project\utils\helpers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiment_data['date'] = sentiment_data['date'].dt.ceil('T')  # 'T' stands for minute
c:\Users\Mert\OneDrive\Desktop\Courses\FBG_Org\financial-big-data-project\utils\helpers.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

,Source,date,text,Sentiment,Open,Log_Return
1,Crypto_News,2021-11-04 16:32:00+00:00,'Do you think I'm buying anything at all witho...,0,0.2571,-0.017352
2,Crypto_News,2021-11-05 13:37:00+00:00,The jury concluded that none of the products s...,0,0.2630,0.022689
3,Crypto_News,2021-11-05 15:39:00+00:00,Bitcoin (BTC) fell during the day before findi...,-1,0.2598,-0.012242
4,Crypto_News,2021-11-08 14:44:00+00:00,Crypto market sentiment has continued its upwa...,1,0.2761,0.060851
5,Crypto_News,2021-11-08 20:00:00+00:00,Here is our weekly collection of digital asset...,0,0.2815,0.019369


### **Without Parallelization**

In [10]:
# Measure the start time
start_time = time.time()

# Example usage:
transformations = []  # if you have extra steps to do outside the param search
initial_params = {
    'gamma': 0.00015,
    'beta': 0.3,
    'delta': 1,
    'lambda': 30,
    'alpha': 0.1
}

summary_df = rolling_calibration_single_bar_summary(
    data=intermediate_data,
    transformations=transformations,
    initial_params=initial_params,
    gamma_values = [0.0003,0.001],
    beta_values = [0.3,0.7],
    delta_values = [5,10],
    lambda_values = [30,60],
    alpha_values = [0.1,0.5],
    window_size=1000,
    step_size=1,
    train_ratio=0.6,
    val_ratio=0.2,
    test_ratio=0.2,
    optimize_params_func=optimize_params_func
)

# Measure the end time
end_time = time.time()

# Calculate the runtime
run_time = end_time - start_time
print(f"Function runtime: {run_time:.2f} seconds")

Rolling Calibration (One Bar): 100%|██████████| 8278/8278 [4:24:44<00:00,  1.92s/it]  

Function runtime: 15884.42 seconds


### **With parallelization**

In [11]:
#Merged News data
data = read_file("./data/processed/final_df.csv")
#Doge price data
doge_price = read_file("./data/clean/doge_price.parquet").rename({"Date":"date"}, axis = 1)
#Sentiment data is the tesla news sentiment data
sentiment_data = data[data["Source"] == "Crypto_News"]

merged_df = process_and_merge_data_continous_price(sentiment_data, doge_price)

#Calculating log returns and then inspecting the return dataframe 
intermediate_data = calculate_log_returns(merged_df,'Open')
intermediate_data.head()

c:\Users\Mert\OneDrive\Desktop\Courses\FBG_Org\financial-big-data-project\utils\helpers.py:63: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  sentiment_data['date'] = sentiment_data['date'].dt.ceil('T')  # 'T' stands for minute
c:\Users\Mert\OneDrive\Desktop\Courses\FBG_Org\financial-big-data-project\utils\helpers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiment_data['date'] = sentiment_data['date'].dt.ceil('T')  # 'T' stands for minute
c:\Users\Mert\OneDrive\Desktop\Courses\FBG_Org\financial-big-data-project\utils\helpers.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

,Source,date,text,Sentiment,Open,Log_Return
1,Crypto_News,2021-11-04 16:32:00+00:00,'Do you think I'm buying anything at all witho...,0,0.2571,-0.017352
2,Crypto_News,2021-11-05 13:37:00+00:00,The jury concluded that none of the products s...,0,0.2630,0.022689
3,Crypto_News,2021-11-05 15:39:00+00:00,Bitcoin (BTC) fell during the day before findi...,-1,0.2598,-0.012242
4,Crypto_News,2021-11-08 14:44:00+00:00,Crypto market sentiment has continued its upwa...,1,0.2761,0.060851
5,Crypto_News,2021-11-08 20:00:00+00:00,Here is our weekly collection of digital asset...,0,0.2815,0.019369


In [12]:
# Measure the start time
start_time = time.time()

# Example usage:
transformations = []  # if you have extra steps to do outside the param search
initial_params = {
    'gamma': 0.00015,
    'beta': 0.3,
    'delta': 1,
    'lambda': 30,
    'alpha': 0.1
}

summary_df = rolling_calibration_single_bar_summary(
    data=intermediate_data,
    transformations=transformations,
    initial_params=initial_params,
    gamma_values = [0.0003,0.001],
    beta_values = [0.3,0.7],
    delta_values = [5,10],
    lambda_values = [30,60],
    alpha_values = [0.1,0.5],
    window_size=1000,
    step_size=1,
    train_ratio=0.6,
    val_ratio=0.2,
    test_ratio=0.2,
    optimize_params_func=optimize_params_func_with_parallelization
)

# Measure the end time
end_time = time.time()

# Calculate the runtime
run_time = end_time - start_time
print(f"Function runtime: {run_time:.2f} seconds")

Rolling Calibration (One Bar): 100%|██████████| 8278/8278 [45:15<00:00,  3.05it/s]

Function runtime: 2715.39 seconds
